In [ ]:
!pip install beautifulSoup4
!pip install requests
!pip install yfinance
!pip install yahoofinancials

     |████████████████████████████████| 63 kB 1.1 MB/s 
     |████████████████████████████████| 6.4 MB 9.2 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
  Created wheel for yahoofinancials: filename=yahoofinancials-1.6-py3-none-any.whl size=15192 sha256=9f3bf6e23a7a48c87f335bad590cb5cbd2036e6ef6b2b5191593e3c69800a63e
  Stored in directory: /root/.cache/pip/wheels/4b

In [ ]:
from bs4 import BeautifulSoup
import urllib.request,sys,time
import datetime
import requests
from time import sleep

import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import yfinance as yf
from yahoofinancials import YahooFinancials

In [ ]:
#Global constants

_URL = 'https://www.prnewswire.com/news-releases/news-releases-list/'
_HOMEURL = 'https://www.prnewswire.com/'
_MONTH_VALS = {"Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun":6, "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12}

In [ ]:
count = 1
newUrl = ''
newsList = []

### Section 1

Scan (the last two weeks or the last week, upto you) and parse news from “newswire” websit.

In [ ]:
def scrapeData(listing):
  topics = listing.find('a')['href']
  # print('topic :' ,topics)
  link = _HOMEURL+topics
  extractNews = listing.h3
  date, time = getDateTime(listing.h3.small.text.split(","))
  # print('Date:',dateTime)
  heading = extractNews.text.split('\n')[2]
  # print(heading)
  return [heading,date,time,link]

In [ ]:
class LimitReached(Exception):
    ''' 
        Custom Exception class to pause exception on reaching limit
    '''
    pass

In [ ]:
def getDateTime(timeStamp):
    '''Helper function that return date and time of the news post'''
    
    if len(timeStamp) is not 1:
      year = int(timeStamp[1].strip())
      month = _MONTH_VALS[timeStamp[0].split(" ")[0]]
      date = int(timeStamp[0].split(" ")[1])
      # Check if the limit of 14 days has been reached or not
      if (datetime.datetime.today() - datetime.datetime(year, month, date)).days is 14:
        raise LimitReached
          # If yes raise exception to break loop
          # raise LimitReached
      date = datetime.datetime(year, month, date).strftime("%b %d, %Y")
      time = timeStamp[-1].split(" ")[-2]
    else:
      date = datetime.datetime.today().strftime("%b %d, %Y")
      time = timeStamp[0][0:-3]
    return date, time

In [ ]:
while True:
  newUrl = _URL + '?page=' + str(count) + '&pagesize=100'
  
  try:
    page=requests.get(newUrl)
  except Exception as e:  
    error_type, error_obj, error_info = sys.exc_info()  
    print ('ERROR FOR LINK:',_URL)  
    print (error_type, 'Line:', error_info.tb_lineno)
  
  soup = BeautifulSoup(page.text, "html.parser")
  news=soup.find_all('div',attrs={'class':'row arabiclistingcards'})

  try:
    for listing in news:
      newsList.append(scrapeData(listing))
  except LimitReached:
    break
  count += 1
  sleep(2)

In [ ]:
newsList

[['Hard Rock International Upholds \'Save the Planet\' Commitment by "Going Dark" for Earth Hour in Partnership with World Wildlife Fund',
  'Apr 21, 2022',
  '17:50 ',
  'https://www.prnewswire.com//news-releases/hard-rock-international-upholds-save-the-planet-commitment-by-going-dark-for-earth-hour-in-partnership-with-world-wildlife-fund-301530644.html'],
 ["Surge Energy America Announces Moody's Credit Rating Upgrades",
  'Apr 21, 2022',
  '17:50 ',
  'https://www.prnewswire.com//news-releases/surge-energy-america-announces-moodys-credit-rating-upgrades-301530645.html'],
 ['American Cancer Society Awards former CEO of the Ann Arbor YMCA, Cathi Duchon, the 2021 Development Volunteer Award for Excellence',
  'Apr 21, 2022',
  '17:48 ',
  'https://www.prnewswire.com//news-releases/american-cancer-society-awards-former-ceo-of-the-ann-arbor-ymca-cathi-duchon-the-2021-development-volunteer-award-for-excellence-301530643.html'],
 ['HAIVISION ANNOUNCES VOTING RESULTS FROM 2022 ANNUAL MEETIN

In [ ]:
header = ['News','Date','Time','Link']
with open('News.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    writer.writerows(newsList)

In [ ]:
df = pd.read_csv('News.csv')
df.head()

,News,Date,Time,Link
0,Hard Rock International Upholds 'Save the Plan...,"Apr 21, 2022",17:50,https://www.prnewswire.com//news-releases/hard...
1,Surge Energy America Announces Moody's Credit ...,"Apr 21, 2022",17:50,https://www.prnewswire.com//news-releases/surg...
2,American Cancer Society Awards former CEO of t...,"Apr 21, 2022",17:48,https://www.prnewswire.com//news-releases/amer...
3,HAIVISION ANNOUNCES VOTING RESULTS FROM 2022 A...,"Apr 21, 2022",17:43,https://www.prnewswire.com//news-releases/haiv...
4,Oi anuncia expiração e resultados de sua ofert...,"Apr 21, 2022",17:43,https://www.prnewswire.com//news-releases/oi-a...


In [ ]:
df.dropna()

,News,Date,Time,Link
0,Hard Rock International Upholds 'Save the Plan...,"Apr 21, 2022",17:50,https://www.prnewswire.com//news-releases/hard...
1,Surge Energy America Announces Moody's Credit ...,"Apr 21, 2022",17:50,https://www.prnewswire.com//news-releases/surg...
2,American Cancer Society Awards former CEO of t...,"Apr 21, 2022",17:48,https://www.prnewswire.com//news-releases/amer...
3,HAIVISION ANNOUNCES VOTING RESULTS FROM 2022 A...,"Apr 21, 2022",17:43,https://www.prnewswire.com//news-releases/haiv...
4,Oi anuncia expiração e resultados de sua ofert...,"Apr 21, 2022",17:43,https://www.prnewswire.com//news-releases/oi-a...
...,...,...,...,...
9506,Sonata Software comunica el plan de sucesión d...,"Apr 08, 2022",01:16,https://www.prnewswire.com//news-releases/sona...
9507,Arcade Gaming Market: 52% of Growth to Origina...,"Apr 08, 2022",01:10,https://www.prnewswire.com//news-releases/arca...
9508,Ever-Glory To Report Fourth Quarter 2021 Earni...,"Apr 08, 2022",01:00,https://www.prnewswire.com//news-releases/ever...
9509,Sonata Software comunica el plan de sucesión d...,"Apr 08, 2022",00:59,https://www.prnewswire.com//news-releases/sona...


In [ ]:
df[df.isnull().any(1)]

,News,Date,Time,Link
3238,NaN,"Apr 19, 2022",08:03,https://www.prnewswire.com//news-releases/coun...
4034,NaN,"Apr 18, 2022",08:44,https://www.prnewswire.com//news-releases/dies...
5240,NaN,"Apr 14, 2022",09:00,https://www.prnewswire.com//news-releases/infl...
5717,NaN,"Apr 13, 2022",21:30,https://www.prnewswire.com//news-releases/plas...
6287,NaN,"Apr 13, 2022",09:00,https://www.prnewswire.com//news-releases/wegm...
7089,NaN,"Apr 12, 2022",11:30,https://www.prnewswire.com//news-releases/flum...


In [ ]:
df = df[df['News'].notna()]

In [ ]:
newsList = df['News']
newsList

0       Hard Rock International Upholds 'Save the Plan...
1       Surge Energy America Announces Moody's Credit ...
2       American Cancer Society Awards former CEO of t...
3       HAIVISION ANNOUNCES VOTING RESULTS FROM 2022 A...
4       Oi anuncia expiração e resultados de sua ofert...
                              ...                        
9506    Sonata Software comunica el plan de sucesión d...
9507    Arcade Gaming Market: 52% of Growth to Origina...
9508    Ever-Glory To Report Fourth Quarter 2021 Earni...
9509    Sonata Software comunica el plan de sucesión d...
9510    GBG aumenta o foco em produtos globais e cria ...
Name: News, Length: 9505, dtype: object

In [53]:
_SYMBOLS = ['CDR','FHN','STYE','GDNSF','VOLT','TLMD']
stock_news = {}

for symb in _SYMBOLS:
    stock_news[symb] = [newsString for index, newsString in enumerate(newsList) if symb in newsString]

print(stock_news)

{'CDR': [], 'FHN': [], 'STYE': [], 'GDNSF': ['SHAREHOLDER ALERT: Weiss Law Reminds LHCG, NLSN, GDNSF, and VOLT Shareholders About Its Ongoing Investigations', 'SHAREHOLDER ALERT: Weiss Law Reminds POLY, LHCG, NLSN, and GDNSF Shareholders About Its Ongoing Investigations'], 'VOLT': ['SHAREHOLDER ALERT: Weiss Law Reminds LHCG, NLSN, GDNSF, and VOLT Shareholders About Its Ongoing Investigations', 'REVOLT Announces the Return of Highly Impactful Shows This Spring'], 'TLMD': []}
